In [24]:
import json
import pandas as pd
import gzip
import numpy as np
from sklearn.externals import joblib
from github import Github
import gensim

from collections import Counter

In [26]:
events = []
# File from: wget http://data.gharchive.org/2018-07-01-15.json.gz
with gzip.open('2018-07-01-15.json.gz') as f:
    for line in f:
        events.append(json.loads(line.strip()))

In [27]:
Counter((event['type'] for event in events));

In [28]:
issues = [event for event in events if event['type'] == 'IssuesEvent']
comments = [event for event in events if event['type'] == 'IssueCommentEvent']
CreateEvents = [event for event in events if event['type'] == 'CreateEvent']

In [29]:
actors=[]
#Hold a list of all the usernames and repo names
#This is needed in order to pull README.md files
names=[]
## pull all of the createEvents in order to get UserID and Repo name for 
# calls to get Readme Content
CreateEvents = [event for event in events if event['type'] == 'CreateEvent']
for event in range(1,len(CreateEvents)):
    thisEvent= CreateEvents[event]
    actors.append(thisEvent['repo'])
    for name in range(0,len(actors)):
        thisName=actors[name]
        names.append(thisName['name'])

In [30]:
## gets rid of doubles:
names = np.array(names)
names = np.unique(names)

In [68]:
## This code will hopefully create a sparsely encoded corpus of readMe files:
def iter_documents(repo_names):
    """
    Generator: iterate over all relevant documents, yielding one
    document (=list of utf8 tokens) at a time.
    """
    # find all readMe files:
    for readMeFile in range(0,len(repo_names)):
        g = Github("6efe0a63859ff9f96f2e1b829db3525b33d2f533 ")
        try:
            thisRepo=g.get_repo(repo_names[readMeFile])
            readMe=thisRepo.get_contents('README.md')
            readMe = readMe.decoded_content
        except:
            pass
            readMe ="No README.md found."
        # break document into utf8 tokens
        yield gensim.utils.tokenize(readMe, lower=True, errors='ignore')

class ReadMeTxtCorpus(object):
    """
    Iterable: on each iteration, return bag-of-words vectors,
    one vector for each document.
 
    Process one document at a time using generators, never
    load the entire corpus into RAM.
 
    """
    def __init__(self, repo_names):
        self.repo_names = repo_names
        # create dictionary = mapping for documents => sparse vectors
        self.dictionary = gensim.corpora.Dictionary(iter_documents(repo_names))
 
    def __iter__(self):
        """
        Again, __iter__ is a generator => TxtSubdirsCorpus is a streamed iterable.
        """
        for tokens in iter_documents(self.repo_names):
            # transform tokens (strings) into a sparse vector, one at a time
            yield self.dictionary.doc2bow(tokens)
 
# that's it! the streamed corpus of sparse vectors is ready
corpus = ReadMeTxtCorpus(names)

In [ ]:
g = Github("6efe0a63859ff9f96f2e1b829db3525b33d2f533 ")
forks = []
watchers = []
stargazers = []
subscribers = []
for repoName in range(0,len(names)):
    thisRepo=g.get_repo(names[repoName])
    try:
        forks.append(thisRepo.forks_count)
    except:
        forks.append(0)
    try:
        watchers.append(thisRepo.watchers_count)
    except:
        watchers.append(0)
    try:
        stargazers.append(thisRepo.stargazers_count)
    except:
        stargazers.append(0)
    try:
        subscribers.append(thisRepo.subscribers_count)
    except:
        subscribers.append(0)

In [60]:
## create a pandas dataframe holding all the variables for tracking the 
# 'popularity' measures of the repos:
readMesAndPopularity = pd.DataFrame(
    {'username/repoName': names,
     'forks': forks,
     'stargazers': stargazers,
     'subscribers': subscribers,
     'ReadMes': corpus
    })

In [62]:
joblib.dump(readMesAndPopularity, "readMeFiles.pkl")

['readMeFiles.pkl']

In [9]:
readMeFiles = joblib.load("readMeFiles.pkl")

3529